# A Data Analysis on Just the Rides CSV File

Check the following questions:
1. incompleted rides
    -> requested, but canceled
    -> accepted, but canceled
    -> arrived at, but canceled
    -> picked up at, but canceled
    
2. completed rides
    -> all 5 from same ride id, -> mark rides completed, or not column

3. Time of ride (day) - earlymorning, morning, afternoon, evening, late-evening
    -> 
    
4. Time of ride (week)  - weekday, weekend, friday night?
    ->

3. Category type of ride - timely, versus long-distance
    -> ratio of distance to cost vs duration to the cost
##### A2. Ride category - time vs distance

Done on other df

In [8]:
# Necessary libs
import os
import numpy as np
import pandas as pd
import scipy
import sklearn

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import dabest

# custom libs
import sys
sys.path.append("../")
from src.data.rider import Fare, Ride

# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load in Data

In [9]:
# datadir = "../data/interim/"
# fpath = os.path.join(datadir, "driver_v1.csv")
                              
# df = pd.read_csv(fpath)

# print(os.listdir(datadir))

In [10]:
datadir = "../data/raw/"

ridetime_fpath = os.path.join(datadir, "ride_timestamps.csv")
                              
time_df = pd.read_csv(ridetime_fpath)

print(os.listdir(datadir))

['.gitkeep', 'ride_timestamps.csv', 'driver_ids.csv', 'ride_ids.csv']


In [11]:
display(time_df.head())

print("The size of the dataframes are: ", time_df.shape)

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


The size of the dataframes are:  (970405, 3)


In [53]:
# ensure datetime
time_df['timestamp'] = pd.to_datetime(time_df['timestamp'])

##### A1. Incomplete/Complete ride

In [19]:
check_df = time_df.copy()
check_df.set_index("ride_id", drop=True, inplace=True)
check_df.drop(labels=["timestamp"], inplace=True, axis=1)
check_dict = check_df.to_dict('list')

display(check_df.head())
print(len(check_dict))

,event
ride_id,
00003037a262d9ee40e61b5c0718f7f0,requested_at
00003037a262d9ee40e61b5c0718f7f0,accepted_at
00003037a262d9ee40e61b5c0718f7f0,arrived_at
00003037a262d9ee40e61b5c0718f7f0,picked_up_at
00003037a262d9ee40e61b5c0718f7f0,dropped_off_at


1


In [54]:
# check each ride_id as "incompleted", or "completed"
complete_event = [
    "requested_at",
    "accepted_at",
    "arrived_at",
    "picked_up_at",
    "dropped_off_at",
]

complete_rides = {}
num_incomplete_rides = 0

for i, (_id, events) in enumerate(check_dict.items()):
    if not all(x in events for x in complete_event):
        complete_rides[_id] = 0
        num_incomplete_rides += 1
    else:
        complete_rides[_id] = 1

        
print("Number incomplete rides: ", num_incomplete_rides)
print("Total number of rides: ", len(ride_ids))

Number incomplete rides:  0
Total number of rides:  194081


In [7]:
time_df["completed_ride"] = time_df["ride_id"].map(complete_rides)

##### A2. Ride time of day

In [55]:
# time_df.drop(labels="time_of_day", inplace=True, axis=1)

In [56]:
import datetime

early_morn = (3, 7)
morn = (7, 11)
afternoon = (11, 3)
late_afternoon = (3, 7)
early_evening = (7, 11)
late_evening = (11, 3)

hours = time_df['timestamp'].dt.hour

print(hours[0:5])
print(time_df["timestamp"][10])
print(datetime.time(hour=time_df["timestamp"][10].hour))

0     9.0
1     9.0
2     9.0
3     9.0
4    10.0
Name: timestamp, dtype: float64
2016-05-16 15:43:09
15:00:00


In [57]:
print(len(time_df['timestamp'].unique()))

865827


In [71]:
def compute_hr_day(df):
    x = df["timestamp"][0]
    hour=x.to_pydatetime().hour
#     return hour
    return datetime.time(hour)
#     lambda x: datetime.time(hour=x.dt.hour)
    
time_df["hr_of_day"] = time_df.pipe(compute_hr_day)
# time_df["hr_of_day"] = pd.to_datetime(time_df["hr_of_day"])
display(time_df.head())

,ride_id,event,timestamp,hr_of_day,time_of_day
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19,09:00:00,0
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51,09:00:00,0
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31,09:00:00,0
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33,09:00:00,0
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05,09:00:00,0


In [74]:
time_df['time_of_day'] = time_df.groupby(pd.Grouper(key='hr_of_day', freq='4H')).ngroup()
display(time_df.head(20))

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

##### A3. Ride time of week

In [ ]:
df.groupby(pd.TimeGrouper(key='your_time_column', freq='3H')).count()